In [2]:
# | default_exp client

In [2]:
# | export 


import json
from typing import Dict, Any, List
from enum import Enum
from dataclasses import dataclass
from rich.console import Console
from rich.panel import Panel

from agentic.tools.manager import ToolManager
from agentic.core.agent import Agent, AgentConfig
from agentic.configs.loader import get_model_config, get_settings_config, get_tools_config, get_reasoning_config


class RequestComplexity(Enum):
    SIMPLE = "simple"
    MODERATE = "moderate"
    COMPLEX = "complex"


@dataclass
class AnalysisResult:
    complexity: RequestComplexity
    confidence: float
    reasoning: str
    requires_planning: bool
    requires_debate: bool
    suggested_tools: List[str]


class BuddyClient:
    """Production-grade AI assistant with intelligent request routing"""
    
    def __init__(self, model: str = None, base_url: str = None, api_key: str = None):
        # Load configurations
        self.model_config = get_model_config()
        self.settings_config = get_settings_config()
        
        # Initialize tool manager
        self.tool_manager = ToolManager()
        
        # Create agent config with routing instructions
        agent_config = AgentConfig(
            name="BuddyAgent",
            instructions="""You are Buddy, an AI assistant that helps with various tasks using available tools.

IMPORTANT ROUTING RULES:
- For comparison questions, "which is better", "pros and cons", evaluation questions: Use the debate tool
- For complex project requests like "build", "create", "develop", "implement": Use the planner tool  
- For all other requests: Handle directly with available tools (fs_read, fs_write, execute_bash, code_interpreter)

Always choose the most appropriate approach based on the request type.""",
            model=model or self.model_config.get('name'),
            temperature=self.model_config.get('temperature', 0.7),
            max_tokens=self.model_config.get('max_tokens')
        )
        
        # Initialize single Agent for all operations
        self.agent = Agent(config=agent_config)
        
        # Initialize console
        self.console = Console()
        
        # Settings - force streaming to true
        self.auto_approve = self.settings_config.get('auto_approve', False)
        self.stream = True  # Always enable streaming
        self.debug = self.settings_config.get('debug', False)
    
    def analyze_request_complexity(self, request: str) -> AnalysisResult:
        """Analyze request complexity and determine routing strategy"""
        
        analysis_prompt = f"""
        Analyze this user request and determine its complexity level:
        
        Request: "{request}"
        
        Consider these factors:
        1. Number of steps required (1-2: simple, 3-5: moderate, 6+: complex)
        2. Technical complexity and domain expertise needed
        3. Need for decision-making or trade-off analysis
        4. File system operations, code generation, or system commands
        5. Planning and coordination requirements
        6. Debate and multi-perspective analysis needs
        
        IMPORTANT: If the request contains words like "debate", "perspectives", "argue", "pros and cons", "should", "regulated", "analysis", "multiple viewpoints", "structured debate", or asks for decision analysis, set requires_debate=true and requires_planning=false.
        
        Respond with JSON:
        {{
            "complexity": "simple|moderate|complex",
            "confidence": 0.0-1.0,
            "reasoning": "Brief explanation",
            "requires_planning": true/false,
            "requires_debate": true/false,
            "suggested_tools": ["tool1", "tool2"]
        }}
        """
        
        try:
            response = self.agent.run(analysis_prompt, stream=True, max_iterations=1)
            content = response.get("content", "")
            
            # Extract JSON from response
            try:
                data = json.loads(content)
                return AnalysisResult(
                    complexity=RequestComplexity(data.get("complexity", "simple")),
                    confidence=data.get("confidence", 0.5),
                    reasoning=data.get("reasoning", ""),
                    requires_planning=data.get("requires_planning", False),
                    requires_debate=data.get("requires_debate", False),
                    suggested_tools=data.get("suggested_tools", [])
                )
            except (json.JSONDecodeError, ValueError):
                return self._fallback_analysis(request)
                
        except Exception as e:
            self.console.print(f"[red]Analysis error: {e}[/red]")
            return self._fallback_analysis(request)
    
    def _fallback_analysis(self, request: str) -> AnalysisResult:
        """Fallback analysis using simple heuristics"""
        
        # Simple keyword-based analysis
        debate_keywords = ["should", "debate", "pros and cons", "analysis", "perspectives"]
        planning_keywords = ["build", "create", "develop", "implement", "project"]
        
        requires_debate = any(keyword in request.lower() for keyword in debate_keywords)
        requires_planning = any(keyword in request.lower() for keyword in planning_keywords)
        
        if requires_debate:
            complexity = RequestComplexity.MODERATE
        elif requires_planning:
            complexity = RequestComplexity.COMPLEX
        else:
            complexity = RequestComplexity.SIMPLE
        
        return AnalysisResult(
            complexity=complexity,
            confidence=0.7,
            reasoning="Fallback heuristic analysis",
            requires_planning=requires_planning,
            requires_debate=requires_debate,
            suggested_tools=[]
        )
    
    def execute_simple_request(self, request: str, suggested_tools: List[str]) -> Dict[str, Any]:
        """Execute simple requests directly with Agent and tools"""
        
        self.console.print(Panel(
            f"[bold green]Executing simple request with direct tool usage[/bold green]",
            title="[bold yellow]🚀 Simple Execution[/bold yellow]",
            border_style="yellow"
        ))
        
        try:
            # Use Agent directly - it handles tool selection and execution
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "simple"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Simple execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "simple"
            }
    
    def execute_complex_request(self, request: str, analysis: AnalysisResult) -> Dict[str, Any]:
        """Execute complex requests by routing to appropriate agents"""
        
        self.console.print(Panel(
            f"[warning]Complex request detected - routing to appropriate agent[/warning]\n"
            f"Complexity: {analysis.complexity.value}\n"
            f"Confidence: {analysis.confidence:.2f}\n"
            f"Reasoning: {analysis.reasoning}",
            title="🧠 Complex Execution"
        ))
        
        try:
            # Route to debate agent if debate is required
            if analysis.requires_debate:
                return self._execute_debate_phase(request)
            
            # Route to planner for moderate/complex planning tasks
            elif analysis.requires_planning:
                return self._execute_planning_phase(request)
            
            # Fallback to simple execution
            else:
                return self.execute_simple_request(request, analysis.suggested_tools)
                
        except Exception as e:
            self.console.print(f"[bold red]❌ Complex execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "complex"
            }
    
    def _execute_planning_phase(self, request: str) -> Dict[str, Any]:
        """Execute planning phase using planner agent"""
        
        try:
            from agentic.agent.planner.executor import DynamicTaskExecutor
            
            executor = DynamicTaskExecutor(self.agent, self.console)
            result = executor.execute(request)
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Planning and execution completed![/bold green]",
                title="Planning Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "planning"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Planning failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "planning"
            }
    
    def _execute_debate_phase(self, request: str) -> Dict[str, Any]:
        """Execute debate phase using debate agent"""
        
        try:
            from agentic.agent.debater import create_debate
            import asyncio
            
            # Run debate asynchronously
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            try:
                result = loop.run_until_complete(
                    create_debate(
                        topic=f"Analysis: {request}",
                        context=f"User request: {request}",
                        max_rounds=2
                    )
                )
            finally:
                loop.close()
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Debate analysis completed![/bold green]",
                title="Debate Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "debate"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Debate failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "debate"
            }
    
    def process_request(self, request: str) -> Dict[str, Any]:
        """Main request processing pipeline - single step execution"""
        
        self.console.print(Panel(
            f"[bold cyan]Processing:[/bold cyan] {request}",
            title="[bold blue]🤖 Buddy AI[/bold blue]",
            border_style="blue"
        ))
        
        try:
            # Single step execution - Agent decides routing internally
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "direct"
            }
            
        except Exception as e:
            error_msg = f"Request processing failed: {str(e)}"
            self.console.print(f"[red]❌ {error_msg}[/red]")
            return {"success": False, "error": error_msg}
    
    def interactive_session(self):
        """Start interactive chat session"""
        
        self.console.print(Panel(
            "[bold green]Welcome to Buddy AI![/bold green]\n"
            "Type your requests and I'll help you with various tasks.\n"
            "Commands: /quit to exit, /clear to clear history",
            title="🤖 Buddy AI Assistant",
            border_style="green"
        ))
        
        while True:
            try:            
                user_input = input("\n💬 You: ").strip()
                
                if not user_input:
                    continue
                
                # Handle commands
                if user_input.lower() in ['/quit', '/exit', 'quit', 'exit']:
                    self.console.print("[yellow]👋 Goodbye![/yellow]")
                    break
                elif user_input.lower() in ['/clear', 'clear']:
                    self.console.clear()
                    continue
                
                # Process the request
                result = self.process_request(user_input)
                
                if not result.get("success", False):
                    self.console.print(f"[red]❌ Error: {result.get('error', 'Unknown error')}[/red]")
                
            except KeyboardInterrupt:
                self.console.print("\n[yellow]👋 Goodbye![/yellow]")
                break
            except Exception as e:
                self.console.print(f"[red]❌ Session error: {e}[/red]")


def main():
    """Main entry point"""
    try:
        client = BuddyClient()
        client.interactive_session()
    except Exception as e:
        print(f"❌ Failed to start Buddy AI: {e}")


